In [3]:
import pandas as pd
import mysql.connector
from sklearn.preprocessing import LabelEncoder

### Load Data

In [15]:
conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="12345",
        database="employee_db",
        auth_plugin='mysql_native_password'
    )

query = """
    SELECT 
        id as EmployeeNumber,
        age as Age,
        position as Position,
        salary as MonthlyIncome,
        overtime as OverTime,
        total_working_years as TotalWorkingYears,
        years_at_company as YearsAtCompany,
        job_satisfaction as JobSatisfaction,
        marital_status as MaritalStatus,
        department as Department,
        status as Attrition
    FROM employees
    """
df = pd.read_sql(query, conn)
conn.close()

# display(df['Position'])

C:\Users\sworn\AppData\Local\Temp\ipykernel_8580\2365271013.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


###  Map positions to original trained positions (using POSITION_MAPPING)

In [17]:
# Position mapping (new tech positions → original trained positions)
POSITION_MAPPING = {
    # Sales
    'Account Executive': 'Sales Executive',
    'Sales Development Rep': 'Sales Representative',
    'Sales Manager': 'Sales Manager',
    
    # R&D
    'Machine Learning Engineer': 'Research Scientist',
    'Data Scientist': 'Research Scientist',
    'Software Engineer': 'Laboratory Technician',  # Closest match
    'DevOps Engineer': 'Laboratory Technician',
    'AI Researcher': 'Research Director',

    # Product/UX
    'Product Manager': 'Research Scientist',
    'UX Designer': 'Laboratory Technician',
    'Product Designer': 'Laboratory Technician',
    
    # HR
    'HR Manager': 'HR Manager',
    'Talent Acquisition': 'HR Specialist',

    # Default fallback
    'DEFAULT': 'Research Scientist'
}

df['JobRole'] = df['Position'].map(lambda x: POSITION_MAPPING.get(x, POSITION_MAPPING['DEFAULT']))

# display(df['JobRole'])

### Encode categorical variables numerically

In [19]:
encoders = {}
categorical_cols = ['OverTime', 'MaritalStatus', 'Department']
for col in categorical_cols:
    encoders[col] = LabelEncoder()
    df[col] = encoders[col].fit_transform(df[col])

display(df.head())

,EmployeeNumber,Age,Position,MonthlyIncome,OverTime,TotalWorkingYears,YearsAtCompany,JobSatisfaction,MaritalStatus,Department,Attrition,JobRole
0,1,52,Product Manager,111246.0,1,2,2,1,1,2,Active,Research Scientist
1,2,32,HR Manager,29298.0,0,23,18,2,0,1,Active,HR Manager
2,3,25,DevOps Engineer,82817.0,1,3,1,4,0,0,Left,Laboratory Technician
3,4,44,UX Designer,24550.0,1,15,2,4,2,2,Left,Laboratory Technician
4,5,35,Account Executive,62873.0,0,26,16,3,1,4,Left,Sales Executive
